In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph
from sparse_util import dilate, zhangSuen

In [165]:
date0='0709_1157'
date1='0709_1557'
date2='0709_1934'
date3='0709_2357'
plate=13

In [7]:
mat_skel=read_mat(get_path(date1,plate,True))['skel']
dic_skel=dic_to_sparse(mat_skel)
skeleton_doc1=sparse.dok_matrix(dic_skel)
skeleton_doc_1_dilated=dilate(skeleton_doc1)
skeleton_doc_1_thinned=zhangSuen(skeleton_doc_1_dilated)
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [13]:
skeleton_doc_1_dilated=dilate(skeleton_doc1)

In [14]:
skeleton_doc_1_thinned=zhangSuen(skeleton_doc_1_dilated)

1989028 1982302
253320 872
245 14
3 4
1 1
0 0


In [114]:
sparse.save_npz(f'Data/skeleton_{date1}_{plate}_full',sparse.csc_matrix(skeleton_doc_1_thinned))

In [115]:
skeleton_doc_1_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date1}_{plate}_full.npz'))

In [7]:
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [12]:
anx_graph_t0m1=prune_graph(nx_graph_1)

In [ ]:
mat_skel=read_mat(get_path(date2,plate,True))['skel']
dic_skel=dic_to_sparse(mat_skel)
skeleton_doc2=sparse.dok_matrix(dic_skel)
skeleton_doc_2_dilated=dilate(skeleton_doc2)
skeleton_doc_2_thinned=zhangSuen(skeleton_doc_2_dilated)
graph_tab_2_reconnected=from_sparse_to_graph(skeleton_doc_2_thinned)
nx_graph_2,pos_2=generate_nx_graph(graph_tab_2_reconnected)
anx_graph_t0=prune_graph(nx_graph_2)

In [116]:
sparse.save_npz(f'Data/skeleton_{date2}_{plate}_full',sparse.csc_matrix(skeleton_doc_2_thinned))

In [117]:
skeleton_doc_2_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date2}_{plate}_full.npz'))

In [166]:
mat_skel=read_mat(get_path(date0,plate,True))['skel']
dic_skel=dic_to_sparse(mat_skel)
skeleton_doc0=sparse.dok_matrix(dic_skel)
skeleton_doc_0_dilated=dilate(skeleton_doc0)
skeleton_doc_0_thinned=zhangSuen(skeleton_doc_0_dilated)
graph_tab_0_reconnected=from_sparse_to_graph(skeleton_doc_0_thinned)
nx_graph_0,pos_0=generate_nx_graph(graph_tab_0_reconnected)
anx_graph_t0m2=prune_graph(nx_graph_0)

1983304 1976721
251952 1280
544 73
44 28
0 0


In [167]:
sparse.save_npz(f'Data/skeleton_{date0}_{plate}_full',sparse.csc_matrix(skeleton_doc_0_thinned))

In [168]:
skeleton_doc_0_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date0}_{plate}_full.npz'))

In [143]:
def find_dense_network(skeleton,crop=500):
    densities=[]
    for pixel in skeleton.keys():
        density=len(skeleton[max(pixel[0]-crop,0):pixel[0]+crop,max(pixel[1]-crop,0):pixel[1]+crop].keys())
        if density>=1000:
            densities.append(density)
            return(skeleton[max(pixel[0]-crop,0):pixel[0]+crop,max(pixel[1]-crop,0):pixel[1]+crop],pixel)

In [186]:
crop=500
skeleton1_dense,pixel=find_dense_network(skeleton_doc_1_thinned)
skeleton2_dense=skeleton_doc_2_thinned[max(pixel[0]-crop,0):pixel[0]+crop,max(pixel[1]-crop,0):pixel[1]+crop]
skeleton0_dense=skeleton_doc_0_thinned[max(pixel[0]-crop,0):pixel[0]+crop,max(pixel[1]-crop,0):pixel[1]+crop]

In [192]:
shift12=shift(skeleton1_dense,skeleton2_dense)['x']

Optimization terminated successfully.
         Current function value: 919.135615
         Iterations: 80
         Function evaluations: 155


In [190]:
shift10=shift(skeleton1_dense,skeleton0_dense)['x']

Optimization terminated successfully.
         Current function value: 1180.052510
         Iterations: 59
         Function evaluations: 113


In [193]:
#careful with shifting over time
shifted_skeleton_doc_2_thinned=shift_skeleton(skeleton_doc_2_thinned,-shift12)
shifted_skeleton_doc_0_thinned=shift_skeleton(skeleton_doc_0_thinned,-shift10)

In [197]:
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,apos_t0m1=generate_nx_graph(graph_tab_1_reconnected)
anx_graph_t0m1=prune_graph(nx_graph_1)
graph_tab_2_reconnected=from_sparse_to_graph(shifted_skeleton_doc_2_thinned)
nx_graph_2,apos_t0=generate_nx_graph(graph_tab_2_reconnected)
anx_graph_t0=prune_graph(nx_graph_2)
graph_tab_0_reconnected=from_sparse_to_graph(shifted_skeleton_doc_0_thinned)
nx_graph_0,apos_t0m2=generate_nx_graph(graph_tab_0_reconnected)
anx_graph_t0m2=prune_graph(nx_graph_0)

In [209]:
apos_t0,anx_graph_t0,anx_graph_t0m1,corresp_tips=second_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)
apos_t0m1,anx_graph_t0m1,anx_graph_t0m2,corresp_tips=second_identification(anx_graph_t0m2,anx_graph_t0m1,apos_t0m2,apos_t0m1,50)

set()


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


set()


In [210]:
begin=15000
end=20000 

In [211]:
skeleton_t0m2=generate_skeleton(anx_graph_t0m2,skeleton_doc_0_thinned.shape)

In [212]:
skeleton_t0m1=generate_skeleton(anx_graph_t0m1,skeleton_doc_1_thinned.shape)

In [213]:
skeleton_t0=generate_skeleton(anx_graph_t0,skeleton_doc_2_thinned.shape)

In [224]:
import pandas as pd
def from_nx_to_tab(nx_graph,pos):
    column_names = ["origin_label","end_label","origin_pos", "end_pos", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_pos=pos[origin_label]
        end_pos = pos[end_label]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],origin_pos)
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], "origin_pos":[origin_pos], "end_pos" : [end_pos], "pixel_list" : [pixel_list]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [227]:
tab_labeled_t0m2 = from_nx_to_tab(anx_graph_t0m2,apos_t0m2)
tab_labeled_t0m1 = from_nx_to_tab(anx_graph_t0m1,apos_t0m1)
tab_labeled_t0 = from_nx_to_tab(anx_graph_t0,apos_t0)

In [234]:
tab_labeled_t0m2.loc[tab_labeled_t0m2['origin_label']==300]

,origin_label,end_label,origin_pos,end_pos,pixel_list
2293,300,340,"[3666, 17790]","[3903, 17995]","[(3666, 17790), (3667, 17790), (3668, 17790), ..."
2294,300,58,"[3666, 17790]","[1841, 18654]","[(3666, 17790), (3665, 17791), (3664, 17792), ..."


In [233]:
tab_labeled_t0m1.loc[tab_labeled_t0m1['origin_label']==300]

,origin_label,end_label,origin_pos,end_pos,pixel_list
2422,300,340,"[3654, 17777]","[3893, 17983]","[(3654, 17777), (3655, 17777), (3656, 17777), ..."
2423,300,58,"[3654, 17777]","[1832, 18640]","[(3654, 17777), (3654, 17778), (3654, 17779), ..."


In [239]:
def get_path(date,plate,skeleton,row=None,column=None,extension=".mat"):
    def get_number(number):
        if number<10:
            return(f'0{number}')
        else:
            return(str(number))
    root_path = r'//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'
    date_plate = f'/2020{date}'
    plate = f'_Plate{plate}'
    if skeleton:
        end='/Analysis/Skeleton'+extension
    else:
        end='/Img'+f'/Img_r{get_number(row)}_c{get_number(column)}.tif'
    return (root_path+date_plate+plate+end)

In [240]:
path = get_path(date0,13,True,extension='_full_labeled.csv')

In [242]:
tab_labeled_t0m2.to_csv(get_path(date0,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0m1.to_csv(get_path(date1,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0.to_csv(get_path(date2,plate,True,extension='_full_labeled.csv'))

In [243]:
graph_t0m2.to_csv(get_path(date0,plate,True,extension='_full.csv'))
graph_t0m1.to_csv(get_path(date0,plate,True,extension='_full.csv'))
graph_t0.to_csv(get_path(date0,plat,True,extension='_full.csv'))

NameError: name 'plat' is not defined

In [235]:
tab_labeled_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full_labeled.csv')
tab_labeled_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full_labeled.csv')
tab_labeled_t0.to_csv(f'Data/graph_{date2}_{plate}_full_labeled.csv')

In [214]:
graph_t0m2=from_sparse_to_graph(skeleton_t0m2)
graph_t0m1=from_sparse_to_graph(skeleton_t0m1)
graph_t0=from_sparse_to_graph(skeleton_t0)

In [215]:
graph_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full.csv')
graph_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full.csv')
graph_t0.to_csv(f'Data/graph_{date2}_{plate}_full.csv')

In [180]:
sub_skeleton_t0m1=skeleton_t0m1[begin:end,begin:end]


In [181]:
sub_skeleton_t0=skeleton_t0[begin:end,begin:end]

In [182]:
list_node_t0m1=[node for node in anx_graph_t0m1 if (begin<pos_1[node][0]<end and begin<pos_1[node][1]<end)]

In [183]:
list_node_t0=[node for node in anx_graph_t0 if (begin<apos_t0[node][0]<end and begin<apos_t0[node][1]<end)]

In [185]:
plot_t_tp1(list_node_t0m1,list_node_t0,pos_1,apos_t0,sub_skeleton_t0m1.todense(),sub_skeleton_t0.todense(),shift=(begin,begin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
plot_t_tp1([],[],pos_1,apos_t0,skeleton1.todense(),skeleton2.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
np.ceil(4.3)

5.0

In [41]:
from scipy.optimize import minimize

In [107]:
sub_skeleton_t0m1.shape

(5000, 5000)

In [172]:
shift_sub_skeleton_t0m1=shift_skeleton(sub_skeleton_t0m1,shift)

In [104]:
np.round(5.4)

5.0

In [40]:
skeleton_t0-skeleton_t0m1

<20687x45556 sparse matrix of type '<class 'numpy.bool_'>'
	with 2683488 stored elements in Compressed Sparse Row format>